# Introduction

In this practice project, you will put the skills acquired through the course to use and create a complete ETL pipeline for accessing data from a website and processing it to meet the requirements.

### Project Scenario:

An international firm that is looking to expand its business in different countries across the world has recruited you. You have been hired as a junior Data Engineer and are tasked with creating an automated script that can extract the list of all countries in order of their GDPs in billion USDs (rounded to 2 decimal places), as logged by the International Monetary Fund (IMF). Since IMF releases this evaluation twice a year, this code will be used by the organization to extract the information as it is updated.

The required data seems to be available on the URL mentioned below:

**URL:** `https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29`

The required information needs to be made accessible as a **``CSV``** file **``Countries_by_GDP.csv``** as well as a table **``Countries_by_GDP``** in a database file **``World_Economies.db``** with attributes Country and **``GDP_USD_billion``**.

Your boss wants you to demonstrate the success of this code by running a query on the database table to display only the entries with more than a 100 billion USD economy. Also, you should log in a file with the entire process of execution named etl_project_log.txt.

You must create a Python code **``'etl_project_gdp.py'``** that performs all the required tasks.

## Objectives
You have to complete the following tasks for this project

1. Write a data extraction function to retrieve the relevant information from the required URL.

2. Transform the available GDP information into 'Billion USD' from 'Million USD'.

3. Load the transformed information to the required CSV file and as a database file.

4. Run the required query on the database.

5. Log the progress of the code with appropriate timestamps.

# Initial setup

Before you start building the code, you need to install the required libraries for it.

The libraries needed for the code are as follows:

1. **``requests``** - The library used for accessing the information from the URL.

2. **``bs4``** - The library containing the **``BeautifulSoup``** function used for webscraping.

3. **``pandas``** - The library used for processing the extracted data, storing it to required formats and communicating with the databases.

4. **``sqlite3``** - The library required to create a database server connection.

5. **``numpy``** - The library required for the mathematical rounding operation as required in the objectives.

6. **``datetime``** - The library containing the function **``datetime``** used for extracting the timestamp for logging purposes.



## Importing the required libraries

In [105]:
import requests as req
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime
import sqlite3 
from bs4 import BeautifulSoup

### Initialize all the known entities

1. **URL:** 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

2. **table_attribs:** The attributes or column names for the dataframe stored as a list. Since the data available in the website is in USD Millions, the attributes should initially be 'Country' and 'GDP_USD_millions'. This will be modified in the transform function later.

3. **db_name:** As mentioned in the Project scenario, ``'World_Economies.db'``

4. **table_name:** As mentioned in the Project scenario, ``'Countries_by_GDP'``

5. **csv_path:** As mentioned in the Project scenario, ``'Countries_by_GDP.csv'``

In [106]:
# Code for ETL operations on Country-GDP data
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
csv_path = 'Countries_by_GDP.csv'
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
conn = sqlite3.connect(db_name)

## Task 1: Extracting information

In [107]:
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. 

    Data Frame Columns:
        Country: The name of the country.
        GDP: Gross domestic product in USD$ (millions).
    
    '''
    # 1. Extract the web page as text.
    html_page = req.get(url).text

    # 2. Parse the text into an HTML object.
    data = BeautifulSoup(html_page, 'html.parser')

    # 3. Create an empty pandas DataFrame named df with columns as the table_attribs.
    df = pd.DataFrame(columns=table_attribs)

    # 4. Extract all 'tbody' attributes of the HTML object and then extract all the rows of the index 2 table using the 'tr' attribute
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')

    for row in rows:
        col = row.find_all('td')
        #print(len(col))
        if len(col) != 0:
            data_dict ={"Country": col[0].get_text(strip = True),
                        "GDP_USD_millions": col[2].get_text(strip = True)}
            df1 = pd.DataFrame(data_dict, index = [0])
            df = pd.concat([df, df1], ignore_index=True )

    return df

#extract(url,table_attribs).info()
#print(extract(url,table_attribs))

## Task 2: Transform information

In [108]:
def transform(df):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''
    
    df["GDP_USD_billions"] = df["GDP_USD_millions"].str.replace(',', '').str.replace('—', '0').astype(float)
    df = df.drop(columns=["GDP_USD_millions"])  

    df["GDP_USD_billions"] = np.round(df["GDP_USD_billions"] / 1000,2)
   
    return df

#transformed_df = transform(extract(url,table_attribs))
#transformed_df.head()

## Task 3: Loading information

In [109]:
def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''

    if len(df) > 0:
        df.to_csv(csv_path)
    else:
        print("Dataframe is empty!")


def load_to_db(df, sql_connection, table_name):
    '''This function saves the final dataframe as a database table
       with the provided name. Function returns nothing.'''
    #sql_connection = db_name
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

#load_to_csv(transformed_df, csv_path)
#load_to_db(transformed_df, conn ,table_name)

## Task 4: Querying the database table

In [110]:
query_smt = """ SELECT * FROM Countries_by_GDP;"""
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

#run_query(query_smt, conn)

## Task 5: Logging progress

In [113]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./etl_project_log.txt","a") as f:
        f.write(timestamp + ' : ' + message + '\n')


## Task 6: Test execution


In [114]:
# Manejo de errores con mensajes de progreso
try:
    log_progress('Preliminaries complete. Initiating ETL process')

    # Extracción de datos
    try:
        df = extract(url, table_attribs)
        log_progress('Data extraction complete. Initiating Transformation process')
    except Exception as e:
        log_progress(f'Error during data extraction: {str(e)}')
        raise

    # Transformación de datos
    try:
        df = transform(df)
        log_progress('Data transformation complete. Initiating loading process')
    except Exception as e:
        log_progress(f'Error during data transformation: {str(e)}')
        raise

    # Carga de datos en CSV
    try:
        load_to_csv(df, csv_path)
        log_progress('Data saved to CSV file')
    except Exception as e:
        log_progress(f'Error saving data to CSV: {str(e)}')
        raise

    # Conexión a la base de datos
    try:
        sql_connection = sqlite3.connect('World_Economies.db')
        log_progress('SQL Connection initiated.')
    except Exception as e:
        log_progress(f'Error initiating SQL connection: {str(e)}')
        raise

    # Carga de datos a la base de datos
    try:
        load_to_db(df, sql_connection, table_name)
        log_progress('Data loaded to Database as table. Running the query')
    except Exception as e:
        log_progress(f'Error loading data to database: {str(e)}')
        raise

    # Ejecutar consulta
    try:
        query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
        run_query(query_statement, sql_connection)
        log_progress('Process Complete.')
    except Exception as e:
        log_progress(f'Error running SQL query: {str(e)}')
        raise

finally:
    # Cierre seguro de la conexión
    try:
        if sql_connection:
            sql_connection.close()
            log_progress('SQL connection closed.')
    except Exception as e:
        log_progress(f'Error closing SQL connection: {str(e)}')

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
          Country  GDP_USD_billions
0           World         105568.78
1   United States          26854.60
2           China          19373.59
3           Japan           4409.74
4         Germany           4308.85
..            ...               ...
65          Kenya            118.13
66         Angola            117.88
67           Oman            104.90
68      Guatemala            102.31
69       Bulgaria            100.64

[70 rows x 2 columns]
